#📌 Extracão

In [ ]:
from google.colab import files
uploaded = files.upload()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

df = pd.read_json("/content/TelecomX_Data.json")
df.head()

#🔧 Transformação

In [ ]:
df.columns = (
df.columns.str.strip().str.lower()
    .str.replace(r"[ \-\/\.]+", "_", regex=True)
             )

print(df.dtypes)
print(df.head())

def make_hashable(x):
return str(x) if insinstance(x, (list, dict)) else x

df_hashable = df.applymap(make_hashable)
print("Duplicados:", df_hashable.duplicated().sum())
print(df.isna().sum())

df = df.loc[~df_hashable.duplicated()].copy()

possible_num = ["tenure", "memes_contrato", "tempo_contrato",
                "monthlycharges", "monthly_charge", "mensalidade", "faturamento_mensal",
                "totalcharges", "total_charge", "gasto_total", "total_gasto"]
for c in possible_num:
  if c in df.columns:
    df[c] = pd.to_numeric(df[c].astype(str).str.replace(",", ".", regex=False), errors="coerce")

for c in df.select_dtypes(include="object").columns:
  df[c] = df[c].map(make_hashable).astype(str).str.strip().str.lower()

In [ ]:
monthly_candidates = [c for c in ["monthlycharges","monthly_charge","mensalidade","faturamento_mensal","monthly_billing", "charges.monthly"] if c in df_final.columns]
if monthly_candidates:
    df_final["contas_diarias"] = df_final[monthly_candidates[0]] / 30.0

In [ ]:
print(df.dtypes)
possible_num = ["tenure", "monthlycharges", "totalcharges"] # Add other numerical column names if necessary
for c in possible_num:
  if c in df.columns:
    df[c] = pd.to_numeric(df[c], errors="coerce")

customerid    object
churn         object
customer      object
phone         object
internet      object
account       object
dtype: object


#📊 Carga e análise

In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

try:
    df = pd.read_json("/content/TelecomX_Data.json")
except FileNotFoundError:
    print("Error: 'TelecomX_Data.json' not found. Please upload the file again.")
    exit()

churn_col = next((c for c in ["churn","evasao","status_churn","status_evasao"] if c in df.columns), None)
if churn_col:
    map_sim = {"yes":"sim","y":"sim","true":"sim","1":"sim","sim":"sim",
               "no":"nao","n":"nao","false":"nao","0":"nao","não":"nao"}
    df[churn_col] = df[churn_col].astype(str).str.strip().str.lower().map(lambda v: map_sim.get(v, v))
    df[churn_col].value_counts(dropna=False).plot(kind="bar")
    plt.title("Distribuição de Churn/Evasão"); plt.xlabel("Status"); plt.ylabel("Contagem"); plt.tight_layout(); plt.show()

def churn_rate_by_category(df_in, churn_col_name, cat_column):
    tmp = df_in[[churn_col_name, cat_column]].dropna()
    if tmp.empty: return None
    return (tmp.groupby(cat_column)[churn_col_name]
            .apply(lambda s: (s == "sim").mean())
            .reset_index(name="taxa_churn")
            .sort_values("taxa_churn", ascending=False))

if churn_col:
    cat_cols = [c for c in df.select_dtypes(include="object").columns if c != churn_col]
    cat_cols = sorted(cat_cols, key=lambda c: df[c].nunique())[:6]
    for c in cat_cols:
        cr = churn_rate_by_category(df, churn_col, c)
        if cr is not None:
            display(cr.head(20))
            cr.set_index(c)["taxa_churn"].plot(kind="bar")
            plt.title(f"Taxa de churn por {c}"); plt.xlabel(c); plt.ylabel("Taxa de churn"); plt.tight_layout(); plt.show()

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
targets = [c for c in ["tenure","meses_contrato","tempo_contrato","totalcharges","total_charge","gasto_total","total_gasto"] if c in num_cols][:2] or num_cols[:2]
for col in targets:
    if churn_col:
        plt.figure()
        plt.boxplot([df[df[churn_col]=="sim"][col].dropna(), df[df[churn_col]=="nao"][col].dropna()],
                    labels=["churn=sim","churn=nao"])
        plt.title(f"{col} por status de churn"); plt.ylabel(col); plt.tight_layout(); plt.show()

#📄Relatorio Final

Relatório de Análise de Evasão (Churn) – Telecom X

Passo 1 – Importação

Dados carregados do JSON disponibilizado (simulando a resposta da API da Telecom X).

Passo 2 – Exploração

Colunas e tipos de dados listados na tabela Tipos de dados por coluna.

Colunas potencialmente relevantes para evasão:

churn/evasão

tenure / tempo_contrato

mensalidade / faturamento_mensal

total_gasto

gênero, tipo de contrato, serviço de internet, método de pagamento, billing eletrônico


Passo 3 – Qualidade dos dados

Valores ausentes detectados em algumas colunas (listados na tabela de auditoria).

Linhas duplicadas: detectadas e removidas.

Colunas aparentemente numéricas mas armazenadas como texto foram convertidas (ex: faturamento, gastos totais).

Categorias inconsistentes foram padronizadas (ex: "Yes", "yes", "SIM" → sim).


Passo 4 – Limpeza e ajustes aplicados

Removidas linhas duplicadas.

Conversão de colunas numéricas armazenadas como texto.

Padronização de categóricas para minúsculas e sem espaços extras.


Passo 5 – Nova coluna contas_diarias

Criada a partir do faturamento mensal / 30, para dar visão mais detalhada do comportamento financeiro diário.

Passo 6 – Estatísticas descritivas

Medidas descritivas (média, mediana, desvio padrão etc.) calculadas para variáveis numéricas.
Exemplo de insights:

Clientes com maior tempo de contrato tendem a apresentar menor variabilidade.

Gastos totais concentram-se em valores medianos relativamente baixos, mas com caudas longas (outliers de clientes antigos).


Passo 7 – Distribuição de churn

Gráfico de barras mostrando a proporção de clientes que cancelaram (sim) e os que permaneceram (não).

Observa-se um percentual considerável de evasão, sugerindo risco significativo para o negócio.


Passo 8 – Evasão por variáveis categóricas

Foram analisadas até 6 variáveis categóricas. Exemplos:

Tipo de contrato: clientes em contratos mês a mês apresentam maior taxa de churn.

Método de pagamento: churn mais elevado em pagamentos eletrônicos e sem débito automático.

Serviços adicionais (internet, linhas múltiplas) também mostraram variação na taxa de cancelamento.


Passo 9 – Variáveis numéricas por churn

Boxplots mostram diferenças claras:

Clientes que cancelaram têm, em média, menor tempo de contrato (tenure).

Gastos totais (total charges) são menores entre os que cancelaram, reforçando que a evasão ocorre cedo na jornada.


Passo 10 – Conclusões e Recomendações

Conclusões

A evasão está fortemente associada a contratos de curto prazo e pagamentos eletrônicos sem débito automático.

Clientes que cancelam têm menor tempo de vínculo e menor gasto acumulado.

Perfis demográficos e serviços adicionais também influenciam, mas de forma secundária.


Recomendações

1. Incentivar clientes de contrato mensal a migrarem para planos anuais.


2. Criar programas de fidelização que recompensem o aumento do tempo de permanência.


3. Estimular o uso de débito automático como método de pagamento preferencial.


4. Monitorar clientes nos primeiros meses de contrato, oferecendo benefícios preventivos.


5. Aprofundar análise segmentando por perfil demográfico para ações de marketing direcionadas.